In [ ]:
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# First things first, we will load the images and labels into a dataframe
# We will use the ImageDataGenerator to load the images and labels into a dataframe keeping in mind that our data is not separated into train and test sets

# We will use the ImageDataGenerator to load the images and labels into a dataframe keeping in mind that our data is not separated into train and test sets
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        'data/UAPD final',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

# Plotting the images
plt.figure(figsize=(12, 12))
for i in range(0, 15):
        plt.subplot(5, 3, i+1)
        for X_batch, Y_batch in train_generator:
                image = X_batch[0]
                plt.imshow(image)
                break
plt.tight_layout()
plt.show()

In [ ]:
images = []
labels = []

for i in range(0, 15):
        for X_batch, Y_batch in train_generator:
                image = X_batch[0]
                label = Y_batch[0]
                images.append(image)
                labels.append(label)
                break
        
images = np.array(images)
labels = np.array(labels)

# Create a df
df = pd.DataFrame()
df['images'] = images.tolist()
df['labels'] = labels.tolist()

# We will now split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['images'], df['labels'], test_size=0.2, random_state=42)

# We will now reshape the data
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

# Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# Plotting the loss and accuracy
plt.figure(figsize=(12, 12))
plt.subplot(2, 2, 1)
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.subplot(2, 2, 2)
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()

In [ ]:
def predictor(img_path):
        img = image.load_img(img_path, target_size=(150, 150))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)
        if prediction == 1:
                print('The image is asphalt')
        else:
                print('The image is not asphalt')
predictor('data/UAPD final/asphalt/asphalt_1.jpg')
predictor('data/UAPD final/not_asphalt/not_asphalt_1.jpg')